In [50]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
import datetime


In [51]:
data_gold_usd = pd.read_csv("csv files/XAU_USD.csv")
data_gold_usd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2865 entries, 0 to 2864
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Tarih   2865 non-null   object 
 1   Şimdi   2865 non-null   object 
 2   Açılış  2865 non-null   object 
 3   Yüksek  2865 non-null   object 
 4   Düşük   2865 non-null   object 
 5   Hac.    0 non-null      float64
 6   Fark %  2865 non-null   object 
dtypes: float64(1), object(6)
memory usage: 156.8+ KB


In [52]:
df = data_gold_usd.drop(['Açılış','Yüksek','Düşük','Hac.','Fark %'], axis=1)
df

,Tarih,Şimdi
0,30.12.2022,"1.824,40"
1,29.12.2022,"1.814,89"
2,28.12.2022,"1.804,09"
3,27.12.2022,"1.813,93"
4,26.12.2022,"1.797,61"
...,...,...
2860,06.01.2012,"1.616,85"
2861,05.01.2012,"1.623,00"
2862,04.01.2012,"1.612,40"
2863,03.01.2012,"1.604,14"


In [53]:
df['Şimdi'] = df['Şimdi'].apply(lambda x: float(x.split()[0].replace(',', '')))
df


,Tarih,Şimdi
0,30.12.2022,1.82440
1,29.12.2022,1.81489
2,28.12.2022,1.80409
3,27.12.2022,1.81393
4,26.12.2022,1.79761
...,...,...
2860,06.01.2012,1.61685
2861,05.01.2012,1.62300
2862,04.01.2012,1.61240
2863,03.01.2012,1.60414


## LSTM

In [54]:
# univariate data preparation
from numpy import array

# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-3:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

# define input sequence
raw_seq = df['Şimdi']
# choose a number of time steps
n_steps = 5
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# summarize the data
for i in range(len(X)):
	print(X[i], y[i])

[1.8244  1.81489 1.80409 1.81393 1.79761] 1.79791
[1.81489 1.80409 1.81393 1.79761 1.79791] 1.79257
[1.80409 1.81393 1.79761 1.79791 1.79257] 1.81413
[1.81393 1.79761 1.79791 1.79257 1.81413] 1.81741
[1.79761 1.79791 1.79257 1.81413 1.81741] 1.78744
[1.79791 1.79257 1.81413 1.81741 1.78744] 1.79234
[1.79257 1.81413 1.81741 1.78744 1.79234] 1.77666
[1.81413 1.81741 1.78744 1.79234 1.77666] 1.80727
[1.81741 1.78744 1.79234 1.77666 1.80727] 1.81029
[1.78744 1.79234 1.77666 1.80727 1.81029] 1.78115
[1.79234 1.77666 1.80727 1.81029 1.78115] 1.79663
[1.77666 1.80727 1.81029 1.78115 1.79663] 1.78923
[1.80727 1.81029 1.78115 1.79663 1.78923] 1.78622
[1.81029 1.78115 1.79663 1.78923 1.78622] 1.77091
[1.78115 1.79663 1.78923 1.78622 1.77091] 1.76839
[1.79663 1.78923 1.78622 1.77091 1.76839] 1.79782
[1.78923 1.78622 1.77091 1.76839 1.79782] 1.80289
[1.78622 1.77091 1.76839 1.79782 1.80289] 1.76845
[1.77091 1.76839 1.79782 1.80289 1.76845] 1.74973
[1.76839 1.79782 1.80289 1.76845 1.74973] 1.74075


In [55]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
# define model
model = Sequential()
model.add(LSTM(400, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=200, verbose=1)


Epoch 1/200
90/90 [==============================] - 2s 5ms/step - loss: 0.1995
Epoch 2/200
90/90 [==============================] - 0s 5ms/step - loss: 6.7316e-04
Epoch 3/200
90/90 [==============================] - 1s 6ms/step - loss: 5.8995e-04
Epoch 4/200
90/90 [==============================] - 0s 5ms/step - loss: 5.7445e-04
Epoch 5/200
90/90 [==============================] - 0s 5ms/step - loss: 5.9482e-04
Epoch 6/200
90/90 [==============================] - 0s 5ms/step - loss: 5.9602e-04
Epoch 7/200
90/90 [==============================] - 0s 5ms/step - loss: 5.8013e-04
Epoch 8/200
90/90 [==============================] - 0s 5ms/step - loss: 5.8242e-04
Epoch 9/200
90/90 [==============================] - 0s 5ms/step - loss: 5.7062e-04
Epoch 10/200
90/90 [==============================] - 0s 5ms/step - loss: 6.2805e-04
Epoch 11/200
90/90 [==============================] - 1s 6ms/step - loss: 5.8754e-04
Epoch 12/200
90/90 [==============================] - 0s 5ms/step - loss: 5.78

In [56]:
# demonstrate prediction
x_input = array([[df['Şimdi'][0], df['Şimdi'][1], df['Şimdi'][2], df['Şimdi'][3], df['Şimdi'][4]]])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

[[1.7924862]]


## Daily

In [57]:
x = datetime.datetime.now()
print(f"Prediction: {round(yhat[0][0]*1000,2)} $ Gold/USD in {x.strftime('%x')}") # onluk olarak yukarı yuvarlandı.

Prediction: 1792.49 $ Gold/USD in 01/01/23


## Weekly

In [58]:
x_input = array([df['Şimdi'][0], df['Şimdi'][1], df['Şimdi'][2], df['Şimdi'][3], float(yhat)])
x_input = x_input.reshape((1, n_steps, n_features))
yhat_2 = model.predict(x_input, verbose=0)

x_input = array([df['Şimdi'][0], df['Şimdi'][1], df['Şimdi'][2], float(yhat_2), float(yhat)])
x_input = x_input.reshape((1, n_steps, n_features))
yhat_3 = model.predict(x_input, verbose=0)

x_input = array([df['Şimdi'][0], df['Şimdi'][1],float(yhat_3), float(yhat_2), float(yhat)])
x_input = x_input.reshape((1, n_steps, n_features))
yhat_4 = model.predict(x_input, verbose=0)

x_input = array([df['Şimdi'][0], float(yhat_4),float(yhat_3), float(yhat_2), float(yhat)])
x_input = x_input.reshape((1, n_steps, n_features))
yhat_5 = model.predict(x_input, verbose=0)


yhat = float(yhat)*1000
yhat_2 = float(yhat_2)*1000
yhat_3 = float(yhat_3)*1000
yhat_4 = float(yhat_4)*1000
yhat_5 = float(yhat_5)*1000


In [59]:
# onluk olarak yukarı yuvarlandı.
print(f"Monday: {round(yhat,3)} Tuesday : {round(yhat_2,3)} Wednesday: {round(yhat_3,3)} Thursday: {round(yhat_4,3)} Friday: {round(yhat_5,3)}  $ GOLD/USD ")


Monday: 1792.486 Tuesday : 1787.128 Wednesday: 1789.791 Thursday: 1789.419 Friday: 1789.122  $ GOLD/USD 


## Monthly

In [60]:
import array
exchange = array.array('f', [])

for i in range(len(df)):
    if i % 5 == 0:  # it equals to weekly
        exchange.append(df['Şimdi'][i])


In [61]:
# univariate data preparation
from numpy import array

# split a univariate sequence into samples


def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-3:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)


# define input sequence
raw_seq = exchange
# choose a number of time steps
n_steps = 5
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# summarize the data
for i in range(len(X)):
	print(X[i], y[i])


[1.8244  1.79791 1.79234 1.79663 1.79782] 1.7561399936676025
[1.79791 1.79234 1.79663 1.79782 1.75614] 1.7497400045394897
[1.79234 1.79663 1.79782 1.75614 1.74974] 1.7706899642944336
[1.79663 1.79782 1.75614 1.74974 1.77069] 1.6802699565887451
[1.79782 1.75614 1.74974 1.77069 1.68027] 1.641759991645813
[1.75614 1.74974 1.77069 1.68027 1.64176] 1.6566599607467651
[1.74974 1.77069 1.68027 1.64176 1.65666] 1.641759991645813
[1.77069 1.68027 1.64176 1.65666 1.64176] 1.6945199966430664
[1.68027 1.64176 1.65666 1.64176 1.69452] 1.6596699953079224
[1.64176 1.65666 1.64176 1.69452 1.65967] 1.643090009689331
[1.65666 1.64176 1.69452 1.65967 1.64309] 1.6749399900436401
[1.64176 1.69452 1.65967 1.64309 1.67494] 1.7160500288009644
[1.69452 1.65967 1.64309 1.67494 1.71605] 1.7114399671554565
[1.65967 1.64309 1.67494 1.71605 1.71144] 1.7368600368499756
[1.64309 1.67494 1.71605 1.71144 1.73686] 1.7478300333023071
[1.67494 1.71605 1.71144 1.73686 1.74783] 1.801759958267212
[1.71605 1.71144 1.73686 1.7

In [62]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
# define model
model = Sequential()
model.add(LSTM(500, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=300, verbose=1)
# demonstrate prediction
x_input = array(
    [[exchange[0], exchange[1], exchange[2], exchange[3], exchange[4]]])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)


Epoch 1/300
18/18 [==============================] - 1s 6ms/step - loss: 0.6778
Epoch 2/300
18/18 [==============================] - 0s 6ms/step - loss: 0.0378
Epoch 3/300
18/18 [==============================] - 0s 6ms/step - loss: 0.0096
Epoch 4/300
18/18 [==============================] - 0s 7ms/step - loss: 0.0037
Epoch 5/300
18/18 [==============================] - 0s 6ms/step - loss: 0.0026
Epoch 6/300
18/18 [==============================] - 0s 6ms/step - loss: 0.0023
Epoch 7/300
18/18 [==============================] - 0s 7ms/step - loss: 0.0023
Epoch 8/300
18/18 [==============================] - 0s 7ms/step - loss: 0.0023
Epoch 9/300
18/18 [==============================] - 0s 7ms/step - loss: 0.0023
Epoch 10/300
18/18 [==============================] - 0s 7ms/step - loss: 0.0023
Epoch 11/300
18/18 [==============================] - 0s 6ms/step - loss: 0.0023
Epoch 12/300
18/18 [==============================] - 0s 6ms/step - loss: 0.0023
Epoch 13/300
18/18 [=================

In [63]:
# onluk olarak yukarı yuvarlandı.
print(f"Next Month -> Prediction: {round(float(yhat[0][0]*1000),3)}  $ Gold/USD in {x.strftime('%B')} ")


Next Month -> Prediction: 1798.843  $ Gold/USD in January 


## Yearly

In [64]:
import array
exchange_2 = array.array('f', [])

for i in range(len(df)):
    if i % 30 == 0:  # it equals to weekly
        exchange_2.append(df['Şimdi'][i])


In [65]:
# univariate data preparation
from numpy import array

# split a univariate sequence into samples


def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-3:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)


# define input sequence
raw_seq = exchange_2
# choose a number of time steps
n_steps = 5
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# summarize the data
for i in range(len(X)):
	print(X[i], y[i])


[1.8244  1.74974 1.69452 1.73686 1.70678] 1.8507699966430664
[1.74974 1.69452 1.73686 1.70678 1.85077] 1.9297300577163696
[1.69452 1.73686 1.70678 1.85077 1.92973] 1.9852900505065918
[1.73686 1.70678 1.85077 1.92973 1.98529] 1.7910300493240356
[1.70678 1.85077 1.92973 1.98529 1.79103] 1.7976800203323364
[1.85077 1.92973 1.98529 1.79103 1.79768] 1.81673002243042
[1.92973 1.98529 1.79103 1.79768 1.81673] 1.7498600482940674
[1.98529 1.79103 1.79768 1.81673 1.74986] 1.7791500091552734
[1.79103 1.79768 1.81673 1.74986 1.77915] 1.7867900133132935
[1.79768 1.81673 1.74986 1.77915 1.78679] 1.8803000450134277
[1.81673 1.74986 1.77915 1.78679 1.8803 ] 1.7431000471115112
[1.74986 1.77915 1.78679 1.8803  1.7431 ] 1.733489990234375
[1.77915 1.78679 1.8803  1.7431  1.73349] 1.8265899419784546
[1.78679 1.8803  1.7431  1.73349 1.82659] 1.8376100063323975
[1.8803  1.7431  1.73349 1.82659 1.83761] 1.9007899761199951
[1.7431  1.73349 1.82659 1.83761 1.90079] 1.941499948501587
[1.73349 1.82659 1.83761 1.9

In [66]:
# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))
# define model
model = Sequential()
model.add(LSTM(500, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X, y, epochs=300, verbose=1)
# demonstrate prediction
x_input = array([[exchange_2[0], exchange_2[1], exchange_2[2], exchange_2[3], exchange_2[4]]])
x_input = x_input.reshape((1, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)


Epoch 1/300
3/3 [==============================] - 1s 6ms/step - loss: 1.9186
Epoch 2/300
3/3 [==============================] - 0s 7ms/step - loss: 1.2218
Epoch 3/300
3/3 [==============================] - 0s 6ms/step - loss: 0.6149
Epoch 4/300
3/3 [==============================] - 0s 7ms/step - loss: 0.1155
Epoch 5/300
3/3 [==============================] - 0s 6ms/step - loss: 0.1809
Epoch 6/300
3/3 [==============================] - 0s 7ms/step - loss: 0.1450
Epoch 7/300
3/3 [==============================] - 0s 6ms/step - loss: 0.0204
Epoch 8/300
3/3 [==============================] - 0s 6ms/step - loss: 0.0638
Epoch 9/300
3/3 [==============================] - 0s 6ms/step - loss: 0.0780
Epoch 10/300
3/3 [==============================] - 0s 6ms/step - loss: 0.0417
Epoch 11/300
3/3 [==============================] - 0s 6ms/step - loss: 0.0171
Epoch 12/300
3/3 [==============================] - 0s 6ms/step - loss: 0.0260
Epoch 13/300
3/3 [==============================] - 0s 7ms/st

In [67]:
# onluk olarak yukarı yuvarlandı.
print(f"Next Year -> Prediction: {round(float(yhat[0][0]*1000),3)}  ₺ GOLD/USD in {x.strftime('%Y')} ")


Next Year -> Prediction: 1660.869  ₺ GOLD/USD in 2023 
